# <font color=#c51b8a>VPOD 'Mine-n-Match':</font>
## <font color=#c994c7>Part 1 Objective</font> - Use Species Names from Microspectrophotemetry Data Sheet to Query NCBI for All Related Opsin Sequences  

In [1]:
import os
import re
import datetime
import time
import pandas as pd
import numpy as np
from deepBreaks.preprocessing import read_data
from Bio import Entrez, SeqIO
email = 'sethfrazer@ucsb.edu'

In [2]:
from mnm_scripts.mine_n_match_functions import ncbi_fetch_opsins, merge_accessory_dbs

## <font color=#c51b8a>Load data-tables with all of the species and sequence data from accessory lmax databases</font> 

### <font color=#c994c7>VPOD Single Cell Microspectrophotmetry (SCP) Datatable </font>
### In this case our dataframe does not have full species name in one column so we must create a list by directly combining the genus and species names. Then filter to create a list of all unique species names 


In [3]:
report_dir = './data_sources/lmax'
species_list = []
df_list = []

In [ ]:
scp_file = f'{report_dir}/vpod/scp_raw.tsv'
scp_df = pd.read_csv(scp_file, index_col=0, sep='\t')
scp_df['Full_Species'] = (scp_df['Genus'] + ' ' + scp_df['Species']).to_list()
df_list.append(scp_df)
scp_df.head()

In [5]:
species_list+=scp_df['Full_Species'].to_list()

### <font color=#c994c7>Longcore - 'Animal Photopigments' Datatable</font>

### In this next case our dataframe has the full species name in one column so we can create a list directly and filter to create a list of all unique species names


In [ ]:
longcore_file = f'{report_dir}/longcore_data/AnimalPhotopigmentsV1_1.csv'
longcore_df = pd.read_csv(longcore_file, index_col=0)
df_list.append(longcore_df)
longcore_df.head()

In [7]:
species_list+=longcore_df['Full_Species'].to_list()

### <font color=#c994c7>Murphy and Westerman Datatable</font>

In [ ]:
murphy_westerman_file = f'{report_dir}/murphy_westerman/Murphy and Westerman.csv'
murphy_westerman_df = pd.read_csv(murphy_westerman_file, index_col=0)
df_list.append(murphy_westerman_df)
murphy_westerman_df.head()

In [9]:
species_list+=murphy_westerman_df['Full_Species'].to_list()

### <font color=#c994c7>Caves 'Fish' Datatable</font>

In [ ]:
caves_fish_db_file = f'{report_dir}/caves/caves_fish_db.csv'
caves_df = pd.read_csv(caves_fish_db_file, index_col=0)
caves_df['Full_Species'] = (caves_df['Genus'] + ' ' + caves_df['Species']).to_list()
df_list.append(caves_df)
caves_df.head()

In [11]:
species_list+=caves_df['Full_Species'].to_list()

### <font color=#c994c7>Morgan Porter's Datatables</font>

In [ ]:
porter_file1 = f'{report_dir}/megan_porter/megan_porter_extract_clean_2005.csv'
porter_df1 = pd.read_csv(porter_file1, index_col=0)
porter_df2 = pd.read_csv(f'{report_dir}/megan_porter/porter_2006_table1_clean.tsv', sep='\t', index_col=0)
df_list.append(porter_df1)
df_list.append(porter_df2)
porter_df1.head()

In [ ]:
porter_df2.head()

In [14]:
species_list += (porter_df1['Full_Species'].to_list() + porter_df2['Full_Species'].to_list())

### <font color=#c994c7>Merge Accessory Databases</font>

In [ ]:
# Call the function to merge all the species, lambdamax, and potential accession information into one dataframe
merged_df = merge_accessory_dbs(df_list, report_dir)
merged_df.head()

In [ ]:
merged_df.shape

### <font color=#c994c7>All unique species names have been extracted from accessory databases. Now we iteratively query NCBI for opsins from each species.</font>

In [ ]:
len(species_list)

In [ ]:
species_list = list(set(species_list))
len(species_list)

In [ ]:
ncbi_query_df = ncbi_fetch_opsins(email=email, job_label='mnm_on_all_dbs', out='mnm_on_all_dbs', species_list=species_list[0:5])

## <font color=#c994c7>Part 2 Objective</font> - Match sequence to it's closest MSP value based on OPTICS predictions

We'll need to...

- Query OPTICS with all the sequence data [bootstrap enabled]
- Extract Predictions
- Match to closest MSP value species-by-species [will need a list of the unique species names] // Match to MaxId as the foreign key

In [1]:
import os
import time 
import datetime
import warnings
import pandas as pd
from deepBreaks.preprocessing import read_data
from mnm_scripts.mine_n_match_functions_old import mine_n_match, post_process_matching, get_prots_from_acc

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [ ]:
report_dir = 'mnm_data/mnm_on_mnm_on_all_dbs_2024-12-10_23-31-03' #re-define the report directory if needed
ncbi_q_file = f'./{report_dir}/mnm_on_all_dbs_ncbi_q_data_cleaned.csv'
ncbi = pd.read_csv(ncbi_q_file)
ncbi.head()

In [ ]:
ncbi.shape

In [ ]:
# Example use for OPTICS
# python optics_predictions.py -in msp_mined_seqs.fasta -rd mined_msp_seqs -out mined_seq_predictions.tsv -m wildtype -e aa_prop -b True -ir msp_mined_seq_blastp_report.tsv -r bovine -s False -bsv msp_bs_viz.pdf
pred_dir = 'e:/safra/Documents/GitHub/optics/prediction_outputs/mnm_opsins/optics_on_all_dbs_2024-12-12_11-49-37'
optics_pred_file = f'{pred_dir}/all_dbs_predictions.tsv'
optics = pd.read_csv(optics_pred_file, sep='\t')

optics.head()

In [ ]:
optics.shape

In [ ]:
source_file = './data_sources/lmax/cleaned_vpod_comp_accessory_dbs_2024-12-17_14-17-46.csv'
comp_db = pd.read_csv(source_file)
comp_db.head()

In [ ]:
comp_db.iloc[21]

In [ ]:
final_err_filtered_df = mine_n_match(report_dir, source_file, ncbi_q_file, optics_pred_file, out='vpod_acc_dbs', err_filter = 15)

In [ ]:
final_err_filtered_df.head()

In [ ]:
final_err_filtered_df.shape

In [12]:
# Replace the missing values in the `Accession` column with an empty string
df_2 = comp_db.copy()
df_2

In [ ]:
df_2= df_2[~df_2["Accession"].isna()].reset_index(drop = True)

# Filter the Accession column to include only those without '-' or '–'
df_2 = df_2[~df_2["Accession"].str.contains("-|–")].reset_index(drop = True)

# Create a list of `Accession` in `result_df`
accession_list = final_err_filtered_df["Accession"].to_list()

# Filter `df_2` to include only those whose `Accession` is not in `accession_list`
df_2_filtered = df_2[~df_2["Accession"].isin(accession_list)].reset_index(drop = True)


In [7]:
prot_list = get_prots_from_acc(df_2_filtered['Accession'].to_list())

In [13]:
df_2_filtered['Protein'] = prot_list

In [14]:
# Split the `Full_Species` column by the first space (' ') into `Genus` and `Species`
g_list = [x.split(' ')[0] for x in df_2_filtered['Full_Species']]
sp_list = [' '.join(x.split(' ')[1:]) for x in df_2_filtered['Full_Species']]
sp_list = [' '.join(x.split(' ')[0:]) for x in sp_list]

df_2_filtered["Genus"] = g_list
df_2_filtered["Species"] = sp_list

# Take the `comp_db_id`, `Genus`, `Species`, `Accession`, and `LambdaMax` from the filtered dataframe and rename the `LambdaMax` column to `closest_measurement`
df_2_filtered = df_2_filtered[["comp_db_id", "Genus", "Species", "Accession", "LambdaMax", "Protein"]]

# Add a column named `Notes` with the text 'Known sequence specified in accessory database'
df_2_filtered["Notes"] = "Known sequence specified in accessory database"

# Concatenate this series with the `result_df` dataframe
result_df2 = pd.concat([final_err_filtered_df, df_2_filtered]).reset_index(drop = True)
result_df2.index.name = 'mnm_id'


In [ ]:
result_df2.head()

In [16]:
result_df2.to_csv('mnm_on_vpod_acc_dbs_final_results_fully_filtered_2.csv')